# Proyecto 2
Ariela Mishaan (22052), Alina Carías (22539), Diego Soto (22737), Ignacio Méndez (22613) y Marcos Díaz

In [1]:
import os, json, math, random, time, csv, datetime, warnings, unicodedata, re as _re, sys
from dataclasses import dataclass
from typing import List, Dict, Tuple
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tqdm import tqdm

2025-11-09 17:55:48.844575: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762710949.080150      38 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762710949.153753      38 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
try:
    tqdm.get_lock().locks = []
except Exception:
    pass

plt.style.use('seaborn-v0_8-darkgrid')

In [3]:
# ==== Reporter utilities (versión para Notebook/Kaggle) ====
class Reporter:
    """Reporter para notebook: no redirige sys.stdout; escribe a archivos y deja ver prints en celda."""
    def __init__(self, run_dir="./asl_results", run_name=None):
        ts = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
        self.run_dir = os.path.abspath(run_dir)
        os.makedirs(self.run_dir, exist_ok=True)
        self.run_name = run_name or f"run-{ts}"
        self.run_path = os.path.join(self.run_dir, self.run_name)
        os.makedirs(self.run_path, exist_ok=True)

        self.log_path = os.path.join(self.run_path, "console.log")
        self.metrics_path = os.path.join(self.run_path, "metrics.jsonl")
        self.summary_path = os.path.join(self.run_path, "summary.json")
        self.pred_csv_path = os.path.join(self.run_path, "predictions.csv")

        self._metrics = {}
        print(f"📁 Guardando salida en: {self.run_path}")

    def log(self, msg=""):
            print(msg)
            try:
                with open(self.log_path, "a", encoding="utf-8") as f:
                    f.write(msg + ("\n" if not msg.endswith("\n") else ""))
            except Exception:
                pass
    
    def add_metrics(self, group, **metrics):
        if group not in self._metrics:
            self._metrics[group] = {}
        self._metrics[group].update(metrics)
        with open(self.metrics_path, "a", encoding="utf-8") as f:
            rec = {"group": group, **metrics, "time": datetime.datetime.now().isoformat()}
            f.write(json.dumps(rec, ensure_ascii=False) + "\n")

    def save_predictions(self, rows, fieldnames=("modelo","real","pred")):
        header_needed = not os.path.exists(self.pred_csv_path)
        with open(self.pred_csv_path, "a", newline="", encoding="utf-8") as f:
            w = csv.DictWriter(f, fieldnames=fieldnames)
            if header_needed:
                w.writeheader()
            for r in rows:
                w.writerow({k: r.get(k, "") for k in fieldnames})

    def save_summary(self):
        with open(self.summary_path, "w", encoding="utf-8") as f:
            json.dump(self._metrics, f, indent=2, ensure_ascii=False)

GLOBAL_REPORTER = None

In [4]:
def _find_file(root, filename):
    for dirpath, dirnames, filenames in os.walk(root):
        if filename in filenames:
            return os.path.join(dirpath, filename)
    return None

def autodetect_asl_paths():
    # Intento Kaggle
    kaggle_root = "/kaggle/input"
    candidates = []
    if os.path.exists(kaggle_root):
        for d in os.listdir(kaggle_root):
            cand_dir = os.path.join(kaggle_root, d)
            if not os.path.isdir(cand_dir):
                continue
            meta = _find_file(cand_dir, "supplemental_metadata.csv")
            cmap = _find_file(cand_dir, "character_to_prediction_index.json")
            # Buscar carpeta de landmarks (contiene muchos .parquet)
            landmarks_dir = None
            for sub in os.listdir(cand_dir):
                full = os.path.join(cand_dir, sub)
                if os.path.isdir(full) and "landmark" in sub.lower():
                    # heurística: directorio con muchos .parquet
                    parquet_count = 0
                    for root2, _, files2 in os.walk(full):
                        parquet_count += sum(1 for f in files2 if f.endswith(".parquet"))
                        if parquet_count >= 10:
                            break
                    if parquet_count >= 1:
                        landmarks_dir = full
                        break
            if meta and cmap and landmarks_dir:
                candidates.append((meta, cmap, landmarks_dir))
        if candidates:
            return candidates[0]
    # Intento local (actualiza si lo tienes en otra ruta)
    local_root = os.path.expanduser("~/Downloads/asl-fingerspelling")
    meta = os.path.join(local_root, "supplemental_metadata.csv")
    cmap = os.path.join(local_root, "character_to_prediction_index.json")
    landmarks = os.path.join(local_root, "supplemental_landmarks")
    if os.path.exists(meta) and os.path.exists(cmap) and os.path.exists(landmarks):
        return (meta, cmap, landmarks)
    # Fallback: usa variables de entorno si están definidas
    meta = os.environ.get("ASL_META", meta)
    cmap = os.environ.get("ASL_CMAP", cmap)
    landmarks = os.environ.get("ASL_LANDMARKS", landmarks)
    return (meta, cmap, landmarks)

_detected_meta, _detected_cmap, _detected_landmarks = autodetect_asl_paths()

In [5]:
# ==== Configuración ====
@dataclass
class CFG:
    TRAIN_CSV: str = _detected_meta
    PARQUET_ROOT: str = _detected_landmarks
    CHAR_MAP_JSON: str = _detected_cmap
    OUTPUT_DIR: str = './asl_results'
    RANDOM_STATE: int = 42
    SAMPLE_SEQUENCES: int = 1000
    FRAME_LEN: int = 128
    BATCH_SIZE: int = 8
    EPOCHS: int = 200      # <- ENTRENARÁ 500 ÉPOCAS POR MODELO
    LR: float = 0.01
    DROPOUT: float = 0.0
    NUM_HID: int = 64
    NUM_HEAD: int = 2
    NUM_FEED_FORWARD: int = 128
    NUM_LAYERS_ENC: int = 2
    NUM_LAYERS_DEC: int = 1
    TARGET_MAXLEN: int = 64

CFG = CFG()
CFG.TRAIN_CSV    = "/kaggle/input/asl-fingerspelling/supplemental_metadata.csv"
CFG.CHAR_MAP_JSON= "/kaggle/input/asl-fingerspelling/character_to_prediction_index.json"
CFG.PARQUET_ROOT = "/kaggle/input/asl-fingerspelling/supplemental_landmarks"
os.makedirs(CFG.OUTPUT_DIR, exist_ok=True)

random.seed(CFG.RANDOM_STATE)
tf.random.set_seed(CFG.RANDOM_STATE)
np.random.seed(CFG.RANDOM_STATE)

In [6]:
LPOSE = [13, 15, 17, 19, 21]
RPOSE = [14, 16, 18, 20, 22]
POSE = LPOSE + RPOSE

X = [f'x_right_hand_{i}' for i in range(21)] + [f'x_left_hand_{i}' for i in range(21)] + [f'x_pose_{i}' for i in POSE]
Y = [f'y_right_hand_{i}' for i in range(21)] + [f'y_left_hand_{i}' for i in range(21)] + [f'y_pose_{i}' for i in POSE]
Z = [f'z_right_hand_{i}' for i in range(21)] + [f'z_left_hand_{i}' for i in range(21)] + [f'z_pose_{i}' for i in POSE]
SEL_COLS = X + Y + Z

RHAND_IDX = [i for i, col in enumerate(SEL_COLS) if "right" in col]
LHAND_IDX = [i for i, col in enumerate(SEL_COLS) if "left" in col]
RPOSE_IDX = [i for i, col in enumerate(SEL_COLS) if "pose" in col and int(col.split('_')[-1]) in RPOSE]
LPOSE_IDX = [i for i, col in enumerate(SEL_COLS) if "pose" in col and int(col.split('_')[-1]) in LPOSE]

PAD_TOKEN = 'P'
START_TOKEN = 'S'
END_TOKEN = 'E'


In [7]:
# ==== Helpers ====
_ALLOWED_CHARS = list("abcdefghijklmnopqrstuvwxyz0123456789- '")
def clean_text(s: str) -> str:
    if s is None:
        return ""
    s = unicodedata.normalize("NFKD", s).encode("ascii","ignore").decode("ascii")
    s = s.lower()
    s = _re.sub(r"[^a-z0-9\\- ']", "", s)
    s = _re.sub(r"\\s+", " ", s).strip()
    return s

def levenshtein_distance(s1, s2):
    if len(s1) < len(s2):
        return levenshtein_distance(s2, s1)
    if len(s2) == 0:
        return len(s1)
    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    return previous_row[-1]

In [8]:
class MetricsTracker:
    def __init__(self):
        self.reset()
    def reset(self):
        self.predictions = []
        self.ground_truths = []
        self.losses = []
        self.times = []
    def add_batch(self, preds, truths, loss=None, time_taken=None):
        self.predictions.extend(preds)
        self.ground_truths.extend(truths)
        if loss is not None:
            self.losses.append(loss)
        if time_taken is not None:
            self.times.append(time_taken)
    def calculate_cer(self):
        total_chars = 0
        total_errors = 0
        for pred, truth in zip(self.predictions, self.ground_truths):
            pred = pred.strip()
            truth = truth.strip()
            total_chars += len(truth)
            total_errors += levenshtein_distance(pred, truth)
        return (total_errors / total_chars * 100) if total_chars > 0 else 100.0
    def calculate_wer(self):
        total_words = 0
        total_errors = 0
        for pred, truth in zip(self.predictions, self.ground_truths):
            pred_words = pred.strip().split()
            truth_words = truth.strip().split()
            total_words += len(truth_words)
            total_errors += levenshtein_distance(' '.join(pred_words), ' '.join(truth_words))
        return (total_errors / total_words * 100) if total_words > 0 else 100.0
    def calculate_accuracy(self):
        correct = sum(1 for p, t in zip(self.predictions, self.ground_truths) if p.strip() == t.strip())
        return (correct / len(self.predictions) * 100) if self.predictions else 0.0
    def get_summary(self):
        return {
            'cer': self.calculate_cer(),
            'wer': self.calculate_wer(),
            'accuracy': self.calculate_accuracy(),
            'avg_loss': np.mean(self.losses) if self.losses else 0.0,
            'avg_time': np.mean(self.times) if self.times else 0.0,
            'total_samples': len(self.predictions)
        }

In [9]:
# ==== Preprocesamiento de landmarks ====
def resize_pad(x):
    if tf.shape(x)[0] < CFG.FRAME_LEN:
        x = tf.pad(x, ([[0, CFG.FRAME_LEN - tf.shape(x)[0]], [0, 0], [0, 0]]))
    else:
        x = tf.image.resize(x, (CFG.FRAME_LEN, tf.shape(x)[1]))
    return x

def pre_process(x):
    if not isinstance(x, tf.Tensor):
        x = tf.constant(x, dtype=tf.float32)

    rhand = tf.gather(x, RHAND_IDX, axis=1)
    lhand = tf.gather(x, LHAND_IDX, axis=1)
    rpose = tf.gather(x, RPOSE_IDX, axis=1)
    lpose = tf.gather(x, LPOSE_IDX, axis=1)

    rnan_idx = tf.reduce_any(tf.math.is_nan(rhand), axis=1)
    lnan_idx = tf.reduce_any(tf.math.is_nan(lhand), axis=1)
    rnans = tf.math.count_nonzero(rnan_idx)
    lnans = tf.math.count_nonzero(lnan_idx)

    if rnans > lnans:
        hand = lhand
        pose = lpose
        hand_len = len(LHAND_IDX) // 3
        hand_x = hand[:, 0*hand_len:1*hand_len]
        hand_y = hand[:, 1*hand_len:2*hand_len]
        hand_z = hand[:, 2*hand_len:3*hand_len]
        hand = tf.concat([1.0 - hand_x, hand_y, hand_z], axis=1)
        pose_len = len(LPOSE_IDX) // 3
        pose_x = pose[:, 0*pose_len:1*pose_len]
        pose_y = pose[:, 1*pose_len:2*pose_len]
        pose_z = pose[:, 2*pose_len:3*pose_len]
        pose = tf.concat([1.0 - pose_x, pose_y, pose_z], axis=1)
    else:
        hand = rhand
        pose = rpose

    hand_len = len(LHAND_IDX) // 3
    hand_x = hand[:, 0*hand_len:1*hand_len]
    hand_y = hand[:, 1*hand_len:2*hand_len]
    hand_z = hand[:, 2*hand_len:3*hand_len]
    hand = tf.stack([hand_x, hand_y, hand_z], axis=-1)
    mean = tf.math.reduce_mean(hand, axis=1, keepdims=True)
    std = tf.math.reduce_std(hand, axis=1, keepdims=True)
    hand = (hand - mean) / (std + 1e-8)

    pose_len = len(LPOSE_IDX) // 3
    pose_x = pose[:, 0*pose_len:1*pose_len]
    pose_y = pose[:, 1*pose_len:2*pose_len]
    pose_z = pose[:, 2*pose_len:3*pose_len]
    pose = tf.stack([pose_x, pose_y, pose_z], axis=-1)

    x = tf.concat([hand, pose], axis=1)
    x = resize_pad(x)
    x = tf.where(tf.math.is_nan(x), tf.zeros_like(x), x)
    x = tf.reshape(x, (CFG.FRAME_LEN, -1))
    return x

def read_char_map(path: str):
    with open(path, 'r') as f:
        char_to_num = json.load(f)
    # agrega tokens especiales
    nxt = max(char_to_num.values()) + 1
    char_to_num[PAD_TOKEN] = nxt; nxt += 1
    char_to_num[START_TOKEN] = nxt; nxt += 1
    char_to_num[END_TOKEN] = nxt
    num_to_char = {j: i for i, j in char_to_num.items()}
    return char_to_num, num_to_char

In [10]:
# Cargar mapa de caracteres
CHAR_TO_NUM, NUM_TO_CHAR = read_char_map(CFG.CHAR_MAP_JSON)
VOCAB_SIZE = len(CHAR_TO_NUM)
print(f"📊 Vocabulario: {VOCAB_SIZE} tokens")

def preprocess_phrase(phrase):
    phrase = START_TOKEN + phrase + END_TOKEN
    phrase_tokens = [CHAR_TO_NUM.get(c, 0) for c in phrase]
    if len(phrase_tokens) < CFG.TARGET_MAXLEN:
        phrase_tokens = phrase_tokens + [CHAR_TO_NUM[PAD_TOKEN]] * (CFG.TARGET_MAXLEN - len(phrase_tokens))
    else:
        phrase_tokens = phrase_tokens[:CFG.TARGET_MAXLEN]
    return np.array(phrase_tokens, dtype=np.int32)

def load_parquet_data(file_path: str):
    try:
        df = pd.read_parquet(file_path, columns=SEL_COLS)
        landmarks = df.values.astype(np.float32)
        processed = pre_process(landmarks)
        return processed.numpy()
    except Exception as e:
        return None

def cargar_metadata_un_parquet(csv_path: str, sample_n: int = None):
    print("📂 Cargando metadata...")
    meta = pd.read_csv(csv_path)
    meta = meta.dropna(subset=['path', 'phrase'])
    meta = meta[meta['phrase'].str.len() <= 50]

    parquet_counts = meta['path'].value_counts()
    target_count = sample_n or 100
    suitable_parquets = parquet_counts[parquet_counts >= target_count]
    if len(suitable_parquets) == 0:
        best_parquet = parquet_counts.index[0]
        meta_filtered = meta[meta['path'] == best_parquet]
    else:
        selected_parquet = suitable_parquets.index[0]
        meta_filtered = meta[meta['path'] == selected_parquet]

    if sample_n is not None and sample_n < len(meta_filtered):
        meta_filtered = meta_filtered.sample(sample_n, random_state=CFG.RANDOM_STATE)

    def resolve_path(row):
        p = str(row['path'])
        fname = os.path.basename(p)
        full = os.path.join(CFG.PARQUET_ROOT, fname)
        return full

    meta_filtered = meta_filtered.copy()
    meta_filtered.loc[:, 'abs_path'] = meta_filtered.apply(resolve_path, axis=1)
    print(f"✅ Dataset: {len(meta_filtered)} secuencias")
    return meta_filtered.reset_index(drop=True)

📊 Vocabulario: 62 tokens


In [11]:
class LandmarkEmbedding(layers.Layer):
    def __init__(self, num_hid=64):
        super().__init__()
        self.dense1 = layers.Dense(num_hid, activation="relu")
        self.dropout1 = layers.Dropout(CFG.DROPOUT)
        self.dense2 = layers.Dense(num_hid, activation="relu")
        self.dropout2 = layers.Dropout(CFG.DROPOUT)
        self.num_hid = num_hid

    def call(self, x, training=False):
        if len(x.shape) == 2:
            x = tf.expand_dims(x, 0)
        x = self.dense1(x)
        x = self.dropout1(x, training=training)
        x = self.dense2(x)
        x = self.dropout2(x, training=training)
        seq_len = tf.shape(x)[1]
        positions = tf.range(start=0, limit=seq_len, delta=1, dtype=tf.float32)
        positions = positions[:, tf.newaxis]
        depth = self.num_hid / 2
        depths = tf.range(depth, dtype=tf.float32)[tf.newaxis, :] / depth
        angle_rates = 1 / (10000 ** depths)
        angle_rads = positions * angle_rates
        pos_encoding = tf.concat([tf.sin(angle_rads), tf.cos(angle_rads)], axis=-1)
        pos_encoding = pos_encoding[tf.newaxis, :, :]
        return x + pos_encoding

class TokenEmbedding(layers.Layer):
    def __init__(self, num_vocab=1000, num_hid=64):
        super().__init__()
        self.emb = layers.Embedding(num_vocab, num_hid)
        self.num_hid = num_hid

    def call(self, x):
        x = self.emb(x)
        seq_len = tf.shape(x)[1]
        positions = tf.range(start=0, limit=seq_len, delta=1, dtype=tf.float32)
        positions = positions[:, tf.newaxis]
        depth = self.num_hid / 2
        depths = tf.range(depth, dtype=tf.float32)[tf.newaxis, :] / depth
        angle_rates = 1 / (10000 ** depths)
        angle_rads = positions * angle_rates
        pos_encoding = tf.concat([tf.sin(angle_rads), tf.cos(angle_rads)], axis=-1)
        pos_encoding = pos_encoding[tf.newaxis, :, :]
        return x + pos_encoding

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, num_heads, feed_forward_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim // num_heads, dropout=rate)
        self.ffn = keras.Sequential([
            layers.Dense(feed_forward_dim, activation="relu"),
            layers.Dropout(rate),
            layers.Dense(embed_dim)
        ])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs, training=training)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, num_heads, feed_forward_dim, dropout_rate=0.1):
        super().__init__()
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = layers.LayerNormalization(epsilon=1e-6)
        self.self_att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim // num_heads, dropout=dropout_rate)
        self.enc_att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim // num_heads, dropout=dropout_rate)
        self.self_dropout = layers.Dropout(dropout_rate)
        self.enc_dropout = layers.Dropout(dropout_rate)
        self.ffn_dropout = layers.Dropout(dropout_rate)
        self.ffn = keras.Sequential([
            layers.Dense(feed_forward_dim, activation="relu"),
            layers.Dropout(dropout_rate),
            layers.Dense(embed_dim)
        ])

    def causal_attention_mask(self, batch_size, n_dest, n_src, dtype):
        i = tf.range(n_dest)[:, None]
        j = tf.range(n_src)
        m = i >= j - n_src + n_dest
        mask = tf.cast(m, dtype)
        mask = tf.reshape(mask, [1, n_dest, n_src])
        mult = tf.concat([tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], 0)
        return tf.tile(mask, mult)

    def call(self, enc_out, target, training):
        input_shape = tf.shape(target)
        batch_size = input_shape[0]
        seq_len = input_shape[1]
        causal_mask = self.causal_attention_mask(batch_size, seq_len, seq_len, tf.bool)

        target_att = self.self_att(target, target, attention_mask=causal_mask, training=training)
        target_norm = self.layernorm1(target + self.self_dropout(target_att, training=training))

        enc_out_att = self.enc_att(target_norm, enc_out, training=training)
        enc_out_norm = self.layernorm2(self.enc_dropout(enc_out_att, training=training) + target_norm)

        ffn_out = self.ffn(enc_out_norm)
        ffn_out_norm = self.layernorm3(enc_out_norm + self.ffn_dropout(ffn_out, training=training))
        return ffn_out_norm

class Transformer(keras.Model):
    def __init__(self):
        super().__init__()
        self.loss_metric = keras.metrics.Mean(name="loss")
        self.enc_input = LandmarkEmbedding(num_hid=CFG.NUM_HID)
        self.dec_input = TokenEmbedding(num_vocab=VOCAB_SIZE, num_hid=CFG.NUM_HID)
        self.enc_layers = [TransformerEncoder(CFG.NUM_HID, CFG.NUM_HEAD, CFG.NUM_FEED_FORWARD, rate=CFG.DROPOUT) for _ in range(CFG.NUM_LAYERS_ENC)]
        self.dec_layers = [TransformerDecoder(CFG.NUM_HID, CFG.NUM_HEAD, CFG.NUM_FEED_FORWARD, dropout_rate=CFG.DROPOUT) for _ in range(CFG.NUM_LAYERS_DEC)]
        self.classifier = layers.Dense(VOCAB_SIZE)

    def call(self, inputs, training=False):
        source = inputs[0]
        target = inputs[1]
        x = self.enc_input(source, training=training)
        for enc_layer in self.enc_layers:
            x = enc_layer(x, training=training)
        enc_out = x
        y = self.dec_input(target)
        for dec_layer in self.dec_layers:
            y = dec_layer(enc_out, y, training=training)
        return self.classifier(y)

    @property
    def metrics(self):
        return [self.loss_metric]

    def train_step(self, batch):
        source = batch[0]
        target = batch[1]
        dec_input = target[:, :-1]
        dec_target = target[:, 1:]
        with tf.GradientTape() as tape:
            preds = self([source, dec_input], training=True)
            one_hot = tf.one_hot(dec_target, depth=VOCAB_SIZE)
            mask = tf.math.logical_not(tf.math.equal(dec_target, CHAR_TO_NUM[PAD_TOKEN]))
            loss = self.compiled_loss(one_hot, preds, sample_weight=mask)
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        gradients = [tf.clip_by_norm(g, 1.0) for g in gradients]
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        self.loss_metric.update_state(loss)
        return {"loss": self.loss_metric.result()}


    def generate(self, source, target_start_token_idx, temperature=0.8):
        bs = tf.shape(source)[0]
        enc = self.enc_input(source, training=False)
        for enc_layer in self.enc_layers:
            enc = enc_layer(enc, training=False)
        dec_input = tf.ones((bs, 1), dtype=tf.int32) * target_start_token_idx
        for i in range(CFG.TARGET_MAXLEN - 1):
            dec_out = self.dec_input(dec_input)
            for dec_layer in self.dec_layers:
                dec_out = dec_layer(enc, dec_out, training=False)
            logits = self.classifier(dec_out)
            logits = logits / temperature
            probabilities = tf.nn.softmax(logits[:, -1, :], axis=-1)
            next_token = tf.random.categorical(tf.math.log(probabilities + 1e-10), 1, dtype=tf.int32)
            dec_input = tf.concat([dec_input, next_token], axis=-1)
            if tf.reduce_all(tf.equal(next_token, CHAR_TO_NUM[END_TOKEN])):
                break
        return dec_input



In [12]:
# === Augmentación suave de landmarks (solo para train) ===
def augment_landmarks(arr):
    # arr: (T, D) float32 normalizado
    if np.random.rand() < 0.5:
        arr = arr * np.random.uniform(0.98, 1.02)  # small scale jitter
    if np.random.rand() < 0.5:
        arr = arr + np.random.normal(0, 0.01, size=arr.shape)  # gaussian noise

    # temporal jitter (drop o dup 3% de frames)
    if np.random.rand() < 0.3:
        T = arr.shape[0]
        k = max(1, int(0.03 * T))
        for _ in range(k):
            if np.random.rand() < 0.5 and T > 2:
                # drop
                idx = np.random.randint(0, T)
                arr = np.delete(arr, idx, axis=0)
                T -= 1
            else:
                # duplicate
                idx = np.random.randint(0, T)
                arr = np.insert(arr, idx, arr[idx], axis=0)
                T += 1
        # recortar/pad para mantener longitud fija
        if arr.shape[0] > CFG.FRAME_LEN:
            arr = arr[:CFG.FRAME_LEN]
        elif arr.shape[0] < CFG.FRAME_LEN:
            pad = np.zeros((CFG.FRAME_LEN - arr.shape[0], arr.shape[1]), dtype=arr.dtype)
            arr = np.concatenate([arr, pad], axis=0)

    return arr.astype(np.float32)


In [13]:
def crear_dataset(meta, batch_size, shuffle, cache_path=None):
    def data_generator():
        for idx, row in meta.iterrows():
            try:
                landmarks = load_parquet_data(row['abs_path'])
                if landmarks is not None:
                    # aplicar augmentación solo si shuffle=True (dataset de entrenamiento)
                    if shuffle:
                        landmarks = augment_landmarks(landmarks)
                    phrase_processed = preprocess_phrase(row['phrase'])
                    yield landmarks, phrase_processed
            except Exception:
                continue

    output_signature = (
        tf.TensorSpec(shape=(CFG.FRAME_LEN, len(LHAND_IDX) + len(LPOSE_IDX)), dtype=tf.float32),
        tf.TensorSpec(shape=(CFG.TARGET_MAXLEN,), dtype=tf.int32)
    )

    ds = tf.data.Dataset.from_generator(data_generator, output_signature=output_signature)

    if cache_path is not None:
        ds = ds.cache(cache_path)  # cache en disco antes de mezclar

    if shuffle:
        ds = ds.shuffle(1000)

    ds = ds.batch(batch_size)
    ds = ds.prefetch(tf.data.AUTOTUNE)
    return ds


In [15]:
def train_model(model, train_ds, val_ds, model_name, epochs=CFG.EPOCHS, patience=15, min_delta=0.002, save_best=True):
    print(f"\n{'='*60}\n🏋️ Entrenando modelo: {model_name}\n{'='*60}")
    history = {'train_loss': [], 'val_loss': [], 'epoch_times': [], 'lr': []}

    # Estado para EarlyStopping y mejor checkpoint
    best_val = float('inf')
    best_weights = None
    best_epoch = -1
    no_improve = 0

    for epoch in range(epochs):
        epoch_start = time.time()

        # Imprimir solo una vez al principio cuando se cargan los datos
        if epoch == 0:
            print("📦 Cargando datos... Esto solo se muestra una vez.")

        # ======= ENTRENAMIENTO =======
        train_losses = []
        for batch_idx, batch in enumerate(train_ds):
            loss_dict = model.train_step(batch)
            loss_value = float(loss_dict['loss'].numpy())
            train_losses.append(loss_value)

        avg_train_loss = float(np.mean(train_losses))
        history['train_loss'].append(avg_train_loss)

        # ======= VALIDACIÓN =======
        val_losses = []
        for batch in val_ds:
            source = batch[0]
            target = batch[1]
            dec_input  = target[:, :-1]
            dec_target = target[:,  1:]

            preds = model([source, dec_input], training=False)  # (B, L, V)
            one_hot = tf.one_hot(dec_target, depth=VOCAB_SIZE)
            mask = tf.math.logical_not(tf.math.equal(dec_target, CHAR_TO_NUM[PAD_TOKEN]))
            loss = keras.losses.categorical_crossentropy(one_hot, preds, from_logits=True)
            loss = tf.reduce_sum(loss * tf.cast(mask, tf.float32)) / tf.reduce_sum(tf.cast(mask, tf.float32))
            val_losses.append(float(loss.numpy()))

        avg_val_loss = float(np.mean(val_losses))
        history['val_loss'].append(avg_val_loss)

        # ======= TIEMPO Y LR =======
        epoch_time = time.time() - epoch_start
        history['epoch_times'].append(epoch_time)

        current_lr = model.optimizer.learning_rate
        try:
            current_lr = float(current_lr(model.optimizer.iterations).numpy()) if hasattr(current_lr, '__call__') else float(current_lr.numpy())
        except Exception:
            current_lr = float(CFG.LR)
        history['lr'].append(current_lr)

        # ======= LOG DE ÉPOCA =======
        print(f"\n📈 Época {epoch+1}/{epochs}:")
        print(f"     Train Loss: {avg_train_loss:.4f}")
        print(f"     Val  Loss:  {avg_val_loss:.4f}")
        print(f"     Time:       {epoch_time:.2f}s")
        print(f"     LR:         {current_lr:.2e}")

        # ======= EARLY STOPPING & BEST CHECKPOINT =======
        if avg_val_loss < (best_val - min_delta):
            best_val = avg_val_loss
            best_epoch = epoch
            best_weights = model.get_weights()
            no_improve = 0
            print(f"     🟢 Mejora en val_loss. Nuevo mejor: {best_val:.4f} (época {best_epoch+1})")
        else:
            no_improve += 1
            print(f"     ⚠️  Sin mejora por {no_improve} época(s) (mejor val_loss {best_val:.4f} en la época {best_epoch+1})")

        if no_improve >= patience:
            print(f"🚨 EarlyStopping activado: {no_improve} épocas sin mejora (paciencia={patience}).")
            print(f"    Mejor época: {best_epoch+1} con val_loss={best_val:.4f}")
            break

    # ======= RESTAURAR MEJORES PESOS Y GUARDAR =======
    if best_weights is not None:
        model.set_weights(best_weights)
        print(f"✅ Restaurados pesos de la mejor época: {best_epoch+1} (val_loss={best_val:.4f})")

    if save_best:
        base = f"/kaggle/working/{model_name.replace(' ', '_')}_best"
        try:
            # Formato .keras recomendado y sin optimizador (no necesitas optimizer para inferencia)
            model.save(base + ".keras", include_optimizer=False)
            print(f"💾 Modelo guardado: {base}.keras")
        except Exception as e:
            print(f"⚠️ No se pudo serializar el modelo completo ({e}). Guardo solo pesos.")
            model.save_weights(base + ".weights.h5")
            print(f"💾 Pesos guardados: {base}.weights.h5")


    return history


In [16]:
# === Helper: Beam Search por lote (itera por muestra) ===
def decode_beam_batch(model, source_batch, start_id, end_id, pad_id,
                      beam_width=5, max_len=CFG.TARGET_MAXLEN, length_penalty=0.9):
    """
    Devuelve lista de secuencias (np.int32) decodificadas con beam search.
    - model: Transformer con forward model([source, dec_input], training=False) -> (B, L, V)
    - source_batch: tensor (B, T, D)
    """
    import numpy as np

    def beam_search_single(source_b):
        # Cada haz: (seq[list[int]], logprob[float], finished[bool])
        beams = [([start_id], 0.0, False)]
        for _ in range(max_len - 1):
            candidates = []
            all_finished = True
            for seq, logp, finished in beams:
                if finished:
                    candidates.append((seq, logp, True))
                    continue
                all_finished = False
                dec_in = tf.constant([seq], dtype=tf.int32)            # (1, L_dec)
                src_in = tf.expand_dims(source_b, 0)                   # (1, T, D)
                logits = model([src_in, dec_in], training=False)       # (1, L_dec, V)
                next_logits = logits[:, -1, :]                         # (1, V)
                log_probs = tf.nn.log_softmax(next_logits, axis=-1).numpy()[0]  # (V,)

                # top-k por log_prob
                k = min(beam_width, log_probs.shape[0])
                topk_idx = np.argpartition(-log_probs, k-1)[:k]
                # ordenar esos k por prob
                topk_idx = topk_idx[np.argsort(-log_probs[topk_idx])]

                for idx in topk_idx:
                    new_seq = seq + [int(idx)]
                    new_logp = logp + float(log_probs[idx])
                    candidates.append((new_seq, new_logp, idx == end_id))

            # Seleccionar los mejores 'beam_width' con penalización de longitud
            def score_fn(seq, logp):
                L = max(1, len(seq))
                return logp / (((5.0 + L) / 6.0) ** length_penalty)

            candidates.sort(key=lambda x: score_fn(x[0], x[1]), reverse=True)
            beams = candidates[:beam_width]

            # Si todos terminaron, cortamos
            if all(x[2] for x in beams):
                break

        # Mejor secuencia final
        best_seq = beams[0][0]
        # quitar <START> y truncar en <END>
        out = []
        for tid in best_seq[1:]:
            if tid == end_id:
                break
            if tid != pad_id:
                out.append(tid)
        # pad/trunc a longitud objetivo
        out = out[:CFG.TARGET_MAXLEN]
        if len(out) < CFG.TARGET_MAXLEN:
            out += [pad_id] * (CFG.TARGET_MAXLEN - len(out))
        return np.array(out, dtype=np.int32)

    # Procesar batch
    source_np = source_batch  # tensor (B, T, D)
    B = int(tf.shape(source_np)[0])
    decoded = [beam_search_single(source_np[i]) for i in range(B)]
    return np.stack(decoded, axis=0)  # (B, L)


In [17]:
def evaluate_model(model, val_ds, val_meta, model_name, temperature=0.8):
    global GLOBAL_REPORTER
    print(f"\n{'='*60}\n🔍 Evaluando modelo: {model_name}\n{'='*60}")
    tracker = MetricsTracker()

    START_ID = CHAR_TO_NUM[START_TOKEN]
    END_ID   = CHAR_TO_NUM[END_TOKEN]
    PAD_ID   = CHAR_TO_NUM[PAD_TOKEN]

    # Evaluación por lotes usando Beam Search
    for batch_idx, batch in enumerate(val_ds):
        batch_start = time.time()
        source = batch[0]
        target = batch[1].numpy()

        # --- Beam Search en vez de greedy/temperature ---
        preds = decode_beam_batch(
            model, source,
            start_id=START_ID, end_id=END_ID, pad_id=PAD_ID,
            beam_width=5, max_len=CFG.TARGET_MAXLEN, length_penalty=0.9
        )
        # preds: (B, L) np.int32
        batch_time = time.time() - batch_start

        batch_preds, batch_truths = [], []
        for i in range(len(target)):
            # Ground truth
            gt_tokens = []
            for idx in target[i, :]:
                if idx == END_ID:
                    break
                if idx not in (PAD_ID, START_ID):
                    gt_tokens.append(NUM_TO_CHAR.get(int(idx), ""))
            ground_truth = "".join(gt_tokens)
            batch_truths.append(ground_truth)

            # Prediction (decodificar ids -> chars)
            pred_tokens = []
            for idx in preds[i, :]:
                ch = NUM_TO_CHAR.get(int(idx), "")
                if ch == END_TOKEN:
                    break
                if ch not in (PAD_TOKEN, START_TOKEN):
                    pred_tokens.append(ch)
            prediction = "".join(pred_tokens)
            batch_preds.append(prediction)

        tracker.add_batch(batch_preds, batch_truths, time_taken=batch_time)

        if len(tracker.predictions) > 0:
            partial_cer = tracker.calculate_cer()
            partial_acc = tracker.calculate_accuracy()
            print(f"Evaluación: CER={partial_cer:.2f}% | Accuracy={partial_acc:.2f}%")

    metrics = tracker.get_summary()
    print(f"\n📊 Resultados de {model_name}:")
    print(f"   CER (Character Error Rate): {metrics['cer']:.2f}%")
    print(f"   WER (Word Error Rate):      {metrics['wer']:.2f}%")
    print(f"   Accuracy (Exact Match):     {metrics['accuracy']:.2f}%")
    print(f"   Avg Loss:                   {metrics['avg_loss']:.4f}")
    print(f"   Avg Time per batch:         {metrics['avg_time']:.3f}s")
    print(f"   Total samples:              {metrics['total_samples']}")

    print(f"\n📝 Ejemplos de predicciones:")
    for i in range(min(5, len(tracker.predictions))):
        print(f"\n   Ejemplo {i+1}:")
        print(f"   Real: '{tracker.ground_truths[i]}'")
        print(f"   Pred: '{tracker.predictions[i]}'")

    # --- Reporter writeback ---
    if GLOBAL_REPORTER is not None:
        try:
            GLOBAL_REPORTER.add_metrics(model_name,
                                        CER=metrics.get('cer', None),
                                        WER=metrics.get('wer', None),
                                        Acc=metrics.get('accuracy', None),
                                        AvgLoss=metrics.get('avg_loss', None),
                                        AvgTimePerBatch=metrics.get('avg_time', None),
                                        TotalSamples=metrics.get('total_samples', None))
            rows = [{"modelo": model_name, "real": gt, "pred": pr}
                    for gt, pr in zip(tracker.ground_truths, tracker.predictions)]
            if rows:
                GLOBAL_REPORTER.save_predictions(rows)
        except Exception as _e:
            print(f"[Reporter] No se pudo guardar métricas/predicciones: {_e}")

    return metrics, tracker


In [18]:
# ====== Visualizaciones ======
def plot_training_comparison(histories, model_names, save_path):
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('Comparación de Entrenamiento entre Modelos', fontsize=16, fontweight='bold')

    # Train loss
    ax = axes[0, 0]
    for history, name in zip(histories, model_names):
        ax.plot(history['train_loss'], marker='o', label=name, linewidth=2)
    ax.set_xlabel('Época', fontsize=12)
    ax.set_ylabel('Training Loss', fontsize=12)
    ax.set_title('Pérdida en Entrenamiento', fontsize=14, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)

    # Val loss
    ax = axes[0, 1]
    for history, name in zip(histories, model_names):
        ax.plot(history['val_loss'], marker='s', label=name, linewidth=2)
    ax.set_xlabel('Época', fontsize=12)
    ax.set_ylabel('Validation Loss', fontsize=12)
    ax.set_title('Pérdida en Validación', fontsize=14, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)

    # epoch times
    ax = axes[1, 0]
    x = np.arange(len(histories[0]['epoch_times']))
    width = 0.35
    for i, (history, name) in enumerate(zip(histories, model_names)):
        ax.bar(x + i*width, history['epoch_times'], width, label=name, alpha=0.7)
    ax.set_xlabel('Época', fontsize=12)
    ax.set_ylabel('Tiempo (segundos)', fontsize=12)
    ax.set_title('Tiempo de Entrenamiento por Época', fontsize=14, fontweight='bold')
    ax.set_xticks(x + width / 2)
    ax.set_xticklabels([f'{i+1}' for i in range(len(histories[0]['epoch_times']))])
    ax.legend()
    ax.grid(True, alpha=0.3, axis='y')

    # LR
    ax = axes[1, 1]
    for history, name in zip(histories, model_names):
        ax.plot(history['lr'], marker='^', label=name, linewidth=2)
    ax.set_xlabel('Época', fontsize=12)
    ax.set_ylabel('Learning Rate', fontsize=12)
    ax.set_title('Evolución del Learning Rate', fontsize=14, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_yscale('log')

    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"✅ Gráfica guardada: {save_path}")
    plt.close()


def plot_metrics_comparison(metrics_list, model_names, save_path):
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    fig.suptitle('Comparación de Métricas de Evaluación', fontsize=16, fontweight='bold')
    metric_names = ['cer', 'wer', 'accuracy']
    metric_labels = ['CER (%)', 'WER (%)', 'Accuracy (%)']
    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']
    for idx, (metric, label, color) in enumerate(zip(metric_names, metric_labels, colors)):
        ax = axes[idx]
        values = [m[metric] for m in metrics_list]
        bars = ax.bar(model_names, values, color=color, alpha=0.7, edgecolor='black', linewidth=2)
        for bar, value in zip(bars, values):
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height, f'{value:.2f}%', ha='center', va='bottom', fontsize=12, fontweight='bold')
        ax.set_ylabel(label, fontsize=12)
        ax.set_title(label, fontsize=14, fontweight='bold')
        ax.grid(True, alpha=0.3, axis='y')
        ax.set_ylim(0, max(values) * 1.2 if values else 1.0)
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"✅ Gráfica guardada: {save_path}")
    plt.close()

def plot_error_analysis(trackers, model_names, save_path):
    fig = plt.figure(figsize=(16, 10))
    gs = fig.add_gridspec(3, 2, hspace=0.3, wspace=0.3)
    fig.suptitle('Análisis Detallado de Errores', fontsize=16, fontweight='bold')

    for idx, (tracker, name) in enumerate(zip(trackers, model_names)):
        errors = []
        for pred, truth in zip(tracker.predictions, tracker.ground_truths):
            error = levenshtein_distance(pred.strip(), truth.strip())
            errors.append(error)

        ax = fig.add_subplot(gs[0, idx])
        ax.hist(errors, bins=20, color=f'C{idx}', alpha=0.7, edgecolor='black')
        ax.set_xlabel('Distancia de Levenshtein', fontsize=11)
        ax.set_ylabel('Frecuencia', fontsize=11)
        ax.set_title(f'{name} - Distribución de Errores', fontsize=12, fontweight='bold')
        ax.grid(True, alpha=0.3, axis='y')
        if errors:
            ax.axvline(np.mean(errors), color='red', linestyle='--', linewidth=2, label=f'Media: {np.mean(errors):.2f}')
            ax.legend()

        ax = fig.add_subplot(gs[1, idx])
        pred_lens = [len(p.strip()) for p in tracker.predictions]
        truth_lens = [len(t.strip()) for t in tracker.ground_truths]
        maxlen = max(truth_lens) if truth_lens else 1
        ax.scatter(truth_lens, pred_lens, alpha=0.5, s=30, c=f'C{idx}')
        ax.plot([0, maxlen], [0, maxlen], 'r--', linewidth=2, label='Ideal')
        ax.set_xlabel('Longitud Real', fontsize=11)
        ax.set_ylabel('Longitud Predicha', fontsize=11)
        ax.set_title(f'{name} - Longitud Predicciones', fontsize=12, fontweight='bold')
        ax.grid(True, alpha=0.3)
        ax.legend()

    ax = fig.add_subplot(gs[2, :])
    metrics_to_compare = {
        'CER': [t.calculate_cer() for t in trackers],
        'WER': [t.calculate_wer() for t in trackers],
        'Accuracy': [t.calculate_accuracy() for t in trackers]
    }

    x = np.arange(len(model_names))
    width = 0.25
    for i, (metric_name, values) in enumerate(metrics_to_compare.items()):
        offset = width * (i - 1)
        bars = ax.bar(x + offset, values, width, label=metric_name, alpha=0.8)
        for bar, value in zip(bars, values):
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height, f'{value:.1f}', ha='center', va='bottom', fontsize=10, fontweight='bold')
    ax.set_xlabel('Modelos', fontsize=12)
    ax.set_ylabel('Porcentaje (%)', fontsize=12)
    ax.set_title('Comparación de Métricas por Modelo', fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(model_names)
    ax.legend()
    ax.grid(True, alpha=0.3, axis='y')

    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"✅ Gráfica guardada: {save_path}")
    plt.close()

In [19]:
# ====== HP Switcher ======
def apply_hparams(hp: dict):
    if "NUM_HID" in hp:          CFG.NUM_HID = hp["NUM_HID"]
    if "NUM_HEAD" in hp:         CFG.NUM_HEAD = hp["NUM_HEAD"]
    if "NUM_FEED_FORWARD" in hp: CFG.NUM_FEED_FORWARD = hp["NUM_FEED_FORWARD"]
    if "NUM_LAYERS_ENC" in hp:   CFG.NUM_LAYERS_ENC = hp["NUM_LAYERS_ENC"]
    if "NUM_LAYERS_DEC" in hp:   CFG.NUM_LAYERS_DEC = hp["NUM_LAYERS_DEC"]
    if "DROPOUT" in hp:          CFG.DROPOUT = hp["DROPOUT"]

In [20]:
# === Scheduler: Warmup + Cosine decay (serializable) ===
import math
import tensorflow as tf
from tensorflow import keras

class WarmupCosine(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, lr_max=1e-4, total_steps=100000, warmup_ratio=0.1, lr_min=1e-6):
        super().__init__()
        self.lr_max = tf.cast(lr_max, tf.float32)
        self.lr_min = tf.cast(lr_min, tf.float32)
        self.total_steps = tf.cast(total_steps, tf.float32)
        self.warmup_steps = tf.cast(total_steps * warmup_ratio, tf.float32)

    def __call__(self, step):
        step = tf.cast(step, tf.float32)
        lr_warm = self.lr_max * (step / tf.maximum(1.0, self.warmup_steps))
        progress = tf.clip_by_value(
            (step - self.warmup_steps) / tf.maximum(1.0, (self.total_steps - self.warmup_steps)),
            0.0, 1.0
        )
        lr_cos = self.lr_min + 0.5 * (self.lr_max - self.lr_min) * (1.0 + tf.cos(math.pi * progress))
        return tf.where(step < self.warmup_steps, lr_warm, lr_cos)

    def get_config(self):
        # Devuelve tipos nativos para que Keras pueda serializar
        return {
            "lr_max": float(self.lr_max.numpy()),
            "total_steps": int(self.total_steps.numpy()),
            "warmup_ratio": float((self.warmup_steps.numpy() / max(1.0, self.total_steps.numpy()))),
            "lr_min": float(self.lr_min.numpy()),
        }

    @classmethod
    def from_config(cls, config):
        return cls(**config)


In [21]:
def main():
    global GLOBAL_REPORTER
    print("="*80)
    print("🚀 COMPARACIÓN DE MODELOS: TRANSFORMER-A vs TRANSFORMER-B")
    print("   ASL Fingerspelling Recognition")
    print("="*80)

    print("\n" + "="*80)
    print("📂 FASE 1: CARGA Y PREPARACIÓN DE DATOS")
    print("="*80)
    meta = cargar_metadata_un_parquet(CFG.TRAIN_CSV, sample_n=CFG.SAMPLE_SEQUENCES)
    trn_df, val_df = train_test_split(meta, test_size=0.2, random_state=CFG.RANDOM_STATE)
    print(f"\n✅ Split completado: train={len(trn_df)}, val={len(val_df)}")

    print("\n🔄 Creando datasets...")
    train_ds = crear_dataset(trn_df, CFG.BATCH_SIZE, shuffle=True)
    train_ds = crear_dataset(trn_df, CFG.BATCH_SIZE, shuffle=True,  cache_path='/kaggle/working/train_cache')
    # Cachear validación UNA VEZ y reusarla tanto en "Val" (durante entrenamiento) como en "Evaluación final"
    val_ds   = crear_dataset(val_df, CFG.BATCH_SIZE, shuffle=False)
    val_ds   = val_ds.cache('/kaggle/working/val_cache')  # cache persistente en disco
    val_ds   = val_ds.prefetch(tf.data.AUTOTUNE)

    print("\n" + "="*80)
    print("🤖 FASE 2: ENTRENAMIENTO - TRANSFORMERS (A vs B)")
    print("="*80)

    # --- Dos paquetes de hiperparámetros ---
    HP_A = {
        "NAME": "Transformer-A",
        "NUM_HID": 64,
        "NUM_HEAD": 2,
        "NUM_FEED_FORWARD": 128,
        "NUM_LAYERS_ENC": 2,
        "NUM_LAYERS_DEC": 1,
        "DROPOUT": 0.10,
        "LR": 3e-4,
        "LABEL_SMOOTH": 0.10,
        "TEMP": 0.8,
    }

    HP_B = {
        "NAME": "Transformer-B",
        "NUM_HID": 512,
        "NUM_HEAD": 8,
        "NUM_FEED_FORWARD": 512,
        "NUM_LAYERS_ENC": 4,
        "NUM_LAYERS_DEC": 3,
        "DROPOUT": 0.35,
        "LR": 1e-4,
        "LABEL_SMOOTH": 0.1,
        "TEMP": 0.7,
    }

    # Registrar config
    if GLOBAL_REPORTER is not None:
        try:
            GLOBAL_REPORTER.save_summary()  # inicializa
        except Exception:
            pass

    histories, metrics_list, trackers, names = [], [], [], []

# ---------- ENTRENAR MODELO A ----------
   #

# ---------- ENTRENAR MODELO B ----------
    apply_hparams(HP_B)
    transformer_B = Transformer()

    # Estimar pasos totales (más estable que fijar 1000)
    # Usamos math.ceil para no depender de numpy aquí.
    steps_per_epoch = max(1, math.ceil(len(trn_df) / CFG.BATCH_SIZE))
    # Si prefieres fijar un tope, puedes usar 200; aquí uso las épocas planeadas.
    total_steps = steps_per_epoch * CFG.EPOCHS

    # Scheduler Warmup + Cosine
    lr_schedule_B = WarmupCosine(
        lr_max=HP_B["LR"],      # pico de LR
        total_steps=total_steps,
        warmup_ratio=0.1,      # 10% de warmup
        lr_min=1e-6
    )

    # Optimizer AdamW con weight decay y clipping para estabilizar
    optimizer_B = tf.keras.optimizers.AdamW(
        learning_rate=lr_schedule_B,
        weight_decay=5e-4,
        clipnorm=1.0,
        beta_1=0.9, beta_2=0.98
    )

    transformer_B.compile(
        optimizer=optimizer_B,
        loss=keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=HP_B["LABEL_SMOOTH"])
    )

    hist_B = train_model(transformer_B, train_ds, val_ds, HP_B["NAME"], CFG.EPOCHS)
    metrics_B, tracker_B = evaluate_model(transformer_B, val_ds, val_df, HP_B["NAME"], temperature=HP_B["TEMP"])
    histories = [hist_B]        # Solo el historial de B
    metrics_list = [metrics_B]  # Solo las métricas de B
    trackers = [tracker_B]      # Solo el tracker de B
    names = [HP_B["NAME"]]

    print("\n" + "="*80)
    print("📈 FASE 3: GENERACIÓN DE VISUALIZACIONES")
    print("="*80)
    os.makedirs(CFG.OUTPUT_DIR, exist_ok=True)
    plot_training_comparison(histories, names, os.path.join(CFG.OUTPUT_DIR, 'training_comparison.png'))
    plot_metrics_comparison(metrics_list, names, os.path.join(CFG.OUTPUT_DIR, 'metrics_comparison.png'))
    plot_error_analysis(trackers, names, os.path.join(CFG.OUTPUT_DIR, 'error_analysis.png'))

    print("\n" + "="*80)
    print("🏆 RESUMEN FINAL Y CONCLUSIONES")
    print("="*80)

    print("\n📊 COMPARACIÓN DE MÉTRICAS:")
    print(f"\n{'Métrica':<20} {names[0]:<15}")
    print("-" * 65)
    metrics_to_show = [
        ('CER (%)', 'cer', 'menor'),
        ('WER (%)', 'wer', 'menor'),
        ('Accuracy (%)', 'accuracy', 'mayor'),
        ('Avg Loss', 'avg_loss', 'menor')
    ]
    for metric_name, metric_key, comparison in metrics_to_show:
        b_val = metrics_list[0][metric_key]  # Solo accedemos a las métricas de Modelo B
        print(f"{metric_name:<20} {b_val:<15.2f}")

    print("\n💾 ARCHIVOS GENERADOS:")
    print(f"   📁 Directorio: {CFG.OUTPUT_DIR}")
    print(f"   📊 training_comparison.png")
    print(f"   📊 metrics_comparison.png")
    print(f"   📊 error_analysis.png")

    print("\n" + "="*80)
    print("✅ ANÁLISIS COMPLETADO")
    print("="*80)

# ====== EJECUCIÓN DIRECTA (en notebook) ======
# Crea reporter global (opcional)
GLOBAL_REPORTER = Reporter(run_dir=CFG.OUTPUT_DIR, run_name="run-" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
main()

📁 Guardando salida en: /kaggle/working/asl_results/run-20251109-175757
🚀 COMPARACIÓN DE MODELOS: TRANSFORMER-A vs TRANSFORMER-B
   ASL Fingerspelling Recognition

📂 FASE 1: CARGA Y PREPARACIÓN DE DATOS
📂 Cargando metadata...
✅ Dataset: 1000 secuencias

✅ Split completado: train=800, val=200

🔄 Creando datasets...


I0000 00:00:1762711078.247144      38 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1762711078.248106      38 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5



🤖 FASE 2: ENTRENAMIENTO - TRANSFORMERS (A vs B)

🏋️ Entrenando modelo: Transformer-B
📦 Cargando datos... Esto solo se muestra una vez.

📈 Época 1/200:
     Train Loss: 2.3041
     Val  Loss:  3.7649
     Time:       717.57s
     LR:         5.00e-06
     🟢 Mejora en val_loss. Nuevo mejor: 3.7649 (época 1)

📈 Época 2/200:
     Train Loss: 2.0860
     Val  Loss:  3.1181
     Time:       212.84s
     LR:         1.00e-05
     🟢 Mejora en val_loss. Nuevo mejor: 3.1181 (época 2)

📈 Época 3/200:
     Train Loss: 1.9371
     Val  Loss:  2.9516
     Time:       215.41s
     LR:         1.50e-05
     🟢 Mejora en val_loss. Nuevo mejor: 2.9516 (época 3)

📈 Época 4/200:
     Train Loss: 1.8601
     Val  Loss:  2.9281
     Time:       213.45s
     LR:         2.00e-05
     🟢 Mejora en val_loss. Nuevo mejor: 2.9281 (época 4)

📈 Época 5/200:
     Train Loss: 1.8036
     Val  Loss:  2.9143
     Time:       213.85s
     LR:         2.50e-05
     🟢 Mejora en val_loss. Nuevo mejor: 2.9143 (época 5)

📈 É